In [1]:
%load_ext autoreload
%autoreload 2

import json
import random
import torch
import numpy as np
from PIL import Image, ImageDraw

import sys
sys.path.append("../detr")
import util.misc as utils
from eval import visualize, visualize_2

from main import get_transform, get_model

In [2]:
cmd_args = {
    "data_type": "structure",
    "config_file": "./structure_config.json",
    "model_load_path": "../models/pubtables1m_structure_detr_r18.pth",
    "table_words_dir": "../data/Own-Samples",
}
config_args = json.load(open(cmd_args["config_file"], "rb"))
config_args["device"] = "cpu"

for key, value in cmd_args.items():
    if not key in config_args or not value is None:
        config_args[key] = value
args = type("Args", (object,), config_args)
#print(args.__dict__)

In [3]:
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = args.device
model, criterion, postprocessors = get_model(args, device)

loading model from checkpoint


In [4]:
model.eval();

In [5]:
criterion.eval();

In [6]:
postprocessors

{'bbox': PostProcess()}

In [7]:
target = {
    "img_path": "../data/Own-Samples/Picture5.jpg",
    "boxes":[],
}

img_path = target["img_path"]
img = Image.open(img_path).convert("RGB")
print("Image size:", img.size)

transforms =  get_transform(args.data_type, "val")
img_tensor, target = transforms(img, target)
print("Tensor size:", img_tensor.size())

outputs = model(img_tensor.unsqueeze(0))
#print(outputs.keys())

pred_logits = outputs["pred_logits"][0]
pred_boxes = outputs["pred_boxes"][0]
#visualize(args, target, pred_logits, pred_boxes)


Image size: (1143, 880)
Tensor size: torch.Size([3, 770, 1000])


In [8]:
_, pred_cells, _ = visualize_2(args, target, pred_logits, pred_boxes)

In [9]:
pred_cells[0]

{'bbox': [51.0, 53.0, 269.0, 84.0],
 'column_nums': [0],
 'row_nums': [0],
 'header': True,
 'subcell': False,
 'subheader': False,
 'cell_text': '',
 'spans': []}

In [10]:
print(len(pred_cells))
print(set([d['column_nums'][0] for d in pred_cells]))
print(set([d['row_nums'][0] for d in pred_cells]))
print(set([d['header'] for d in pred_cells]))
print(set([d.get('subcell') for d in pred_cells]))
print(set([d['subheader'] for d in pred_cells]))
print(set([d['cell_text'] for d in pred_cells]))

60
{0, 1, 2, 3, 4, 5, 6}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
{False, True}
{False, None}
{False, True}
{'', '(12)', '3', '56', '(7)', '(70)', '260', '403', 'Trading EBITDA', '(10)', '(32)', '2019', 'profit before exceptional items', '(6)', '(2)', 'Share-based payments', '307', '243', 'Amortisation and epreciation', '320', '337', 'Roadside £m', '(8)', '(1)', 'Insurance £m', '(67)', '(60)', 'Group £m', '316', '284', '304', 'Pension service charge adjustment', '61', '(63)', '71', '2018', '66', '391', 'Operatin', 'Exceptional operating items', 'Operating profit'}


In [15]:
for d in pred_cells:
    if d['cell_text'] in ["2018", "2019", "Roadside £m", "Share-based payments", "Pension service charge adjustment", "Operatin", "(6)"]:
        print(d['cell_text'])
        print(d)

Share-based payments
{'bbox': [51.0, 271.0, 269.0, 323.0], 'column_nums': [0], 'row_nums': [3], 'header': False, 'subcell': False, 'subheader': False, 'cell_text': 'Share-based payments', 'spans': [{'bbox': [61, 271, 197, 290], 'text': 'Share-based', 'flags': 0, 'span_num': 446, 'line_num': 0, 'block_num': 0}, {'bbox': [62, 285, 169, 320], 'text': 'payments', 'flags': 0, 'span_num': 447, 'line_num': 0, 'block_num': 0}]}
Pension service charge adjustment
{'bbox': [51.0, 361.0, 269.0, 412.0], 'column_nums': [0], 'row_nums': [4], 'header': False, 'subcell': False, 'subheader': False, 'cell_text': 'Pension service charge adjustment', 'spans': [{'bbox': [62, 361, 147, 379], 'text': 'Pension', 'flags': 0, 'span_num': 454, 'line_num': 0, 'block_num': 0}, {'bbox': [156, 361, 232, 379], 'text': 'service', 'flags': 0, 'span_num': 455, 'line_num': 0, 'block_num': 0}, {'bbox': [61, 383, 134, 405], 'text': 'charge', 'flags': 0, 'span_num': 456, 'line_num': 0, 'block_num': 0}, {'bbox': [143, 383, 26

In [12]:
#for d in pred_cells:
#    if d.get('spans'):
#        print(d['cell_text'])
#        print(d)